In [29]:
import requests
import json
import pandas as  pd
import time
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

In [72]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [73]:
response.json()

{'signingKey_hex': 'a15ab14f99bcca9435f2bf15daae3dcea373595370f6d989522245b3448d1135',
 'traderId': '2',
 'verifyKey_hex': '39bce380531c2b2738aa6657137eef4669ad9079d696201754426712851f52a3'}

In [74]:
url = 'http://127.0.0.1:5000/createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [75]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [76]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,1,2


In [77]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [78]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [79]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,1,0.551,1,2


In [80]:
# Create a matched trade
url = 'http://127.0.0.1:5000/createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [81]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '2'}

In [82]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 2.8283510208129883 seconds.
matched trade: 1 in 2.7593038082122803 seconds.
matched trade: 2 in 2.725238084793091 seconds.
matched trade: 3 in 2.7483532428741455 seconds.
matched trade: 4 in 2.838487148284912 seconds.
matched trade: 5 in 2.8351478576660156 seconds.
matched trade: 6 in 2.840131998062134 seconds.
matched trade: 7 in 2.85880184173584 seconds.
matched trade: 8 in 2.938283681869507 seconds.
matched trade: 9 in 3.0020751953125 seconds.


In [44]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.2061171531677246 seconds.
market created: 2 in 0.2741429805755615 seconds.
market created: 3 in 0.39956092834472656 seconds.
market created: 4 in 0.7178940773010254 seconds.


In [45]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 1}

In [83]:
url = 'http://127.0.0.1:5000/viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [84]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,0.7
1,2,1,0,0.7
2,3,1,0,0.7
3,4,1,0,0.7


In [85]:
url = 'http://127.0.0.1:5000/viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [86]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,1,1
1,1,1,0.551,-1,1
10,1,1,0.551,1,1
100,1,1,0.551,-1,2
101,1,1,0.551,1,2
102,1,1,0.551,-1,2
103,1,1,0.551,1,2
104,1,1,0.551,1,2
105,1,1,0.551,-1,2
106,1,1,0.551,-1,2


In [96]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.01
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.9246470928192139 seconds.
matched trade: 1 in 0.9676978588104248 seconds.
matched trade: 2 in 0.9386119842529297 seconds.
matched trade: 3 in 0.9572007656097412 seconds.
matched trade: 4 in 0.9613139629364014 seconds.
matched trade: 5 in 0.9193830490112305 seconds.
matched trade: 6 in 0.9202601909637451 seconds.
matched trade: 7 in 0.911531925201416 seconds.
matched trade: 8 in 0.9507687091827393 seconds.
matched trade: 9 in 0.9240102767944336 seconds.


In [103]:
url = 'http://127.0.0.1:5000/viewOpenTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [104]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.01,8,1
1,1,1,0.01,10,2


In [105]:
url = 'http://127.0.0.1:5000/viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [106]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.010,2,1
1,1,1,0.010,-2,2
2,1,1,0.551,0,1
3,1,1,0.551,0,2


In [92]:
tId

'2'